# **Part 2: Agentic Workflows**

![LangChain Logo](figs/AI_Agent.png)

### Imports & Setting up Gemini's API key

In [1]:
import os, getpass, time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.agents import create_agent

from typing import Dict, List, Any
import math


if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
    
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

MODEL_NAME = "gemini-2.5-flash"  # or, feel free to use any LLM here
llm = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0)

/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Task 1: Function and Tool Calling with Agents

In this task, you will explore how modern LLMs can interact with external tools and functions to perform real-world tasks beyond simple text generation.  
You will learn how to **connect an LLM to APIs, databases, and system functions**, enabling it to perform practical operations such as searching the web, managing tasks, and controlling a simulated household device.



#### Overview: What Is Function Calling in LLMs?

**Function calling** allows an LLM to call specific functions or APIs during a conversation based on the user’s request.  
Rather than relying only on natural language generation, the model determines *when and how* to invoke a tool, API, or system function to gather data, perform computations, or change system states.

This mechanism bridges the gap between **language understanding** and **real-world action**.  
It makes the LLM an **active reasoning agent** that can access structured tools or APIs when it detects that such calls are necessary.



#### Why Web Search Tools Matter: Tavily Search API

For this task, we will use the **Tavily Search API** — a web search API designed specifically for LLM applications.  
You can set up Tavily by visiting the official website and generating an API key.

---

## Task Scenario: Household Assistant Application

You will design a small **Household Assistant Application** where an LLM uses function calling to manage everyday tasks.  
The system will include a series of tool functions that the model can invoke based on user prompts.  

The LLM will decide which tool to use and automatically format the function call as defined in its configuration.

---

## Functions to Implement

### 1. Web Search Function

This function will use the **Tavily Search API** to perform web searches when the LLM identifies that external information is needed.  
You should configure your Tavily API key and handle the returned search results.

### 2. To-Do List

Your assistant will maintain a to-do list stored as text embeddings in a vector store (e.g., Chroma).

You'll need a tool for adding to a To-Do List, and a tool to query from that To-Do List

### 3. Get Current Time

A tool to get the current system time

### 4. Toggle Light On/Off

This function simulates turning a light on or off in your home. The current state of the light needs to be stored. 
You also need two tools for this. One to toggle it on (if it's off) or off (if it's on), and the other to check and return the current state.

You can visit the official website and see how to set up the Tavily Search API:

[Tavily Resource](https://docs.tavily.com/documentation/integrations/langchain)

In [2]:
tavily_search = TavilySearch(max_results=5, topic="general")

@tool
def web_search(query: str) -> str:
    """This function searches the Web for the results / answers to a user query. 
    This function is to be invoked if the model cannot accurately answer the question."""
    try:
        results = tavily_search.invoke({"query": query})
        return str(results)
    
    except Exception as e:
        return f"Error performing search: {e}"


In [3]:
bert_model_name = "sentence-transformers/all-MiniLM-L6-v2"

hf_embeddings = HuggingFaceEmbeddings(model_name=bert_model_name)

/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FF5E77A4-1F04-398B-B781-976783A281B8> /opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/nlp-pa4/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan o

In [4]:
# init a vector store (chroma)
todo_vector_store = Chroma(
    collection_name="todo_list",
    embedding_function=hf_embeddings,
    persist_directory="./vectorstores/Part2_Task1_Assistant"
)

@tool
def add_to_todo(task: str) -> str:
    """This function adds a new task to the to-do list."""
    todo_vector_store.add_texts(texts=[task])
    
    return f"Added task: '{task}' to the list."

@tool
def query_todo(query: str = "unfinished tasks") -> str:
    """This function queries the todo list and retrieves top most relevant tasks"""
    results = todo_vector_store.similarity_search(query=query, k=5)
    
    if not results:
        return "The to-do list is empty or no relevant tasks found."
    
    return "\n".join([f"- {doc.page_content}" for doc in results])

@tool
def get_current_time() -> str:
    """This function gets the current time."""
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())


# define a light state
light_toggled = False

@tool
def check_light_state() -> bool:
    """THis function returns the light state"""
    global light_toggled
    return light_toggled

@tool
def toggle_light() -> bool:
    """This functions toggles the light on or off"""
    global light_toggled
    light_toggled = not light_toggled
    return light_toggled

In [5]:
#TODO: Create an Agent Extractor
tools = [web_search, add_to_todo, query_todo, get_current_time, toggle_light, check_light_state]

assistant_system_prompt = """
"You are an intelligent and proactive Household Assistant. You have access to real-world tools to manage the user's home and schedule.

Your Responsibilities:

1) Home Automation: You can control the lights (toggle_light) and check their status (get_light_state).

2) Task Management: You manage a persistent To-Do list. Always use query_todo before adding duplicates, and use add_to_todo to save new tasks.

3) Information: Use web_search for current events or facts, and get_current_time for time-sensitive queries.

Guidelines:

1) Be Efficient: If a user request requires multiple actions (e.g., 'Turn on the lights and find a recipe'), execute all necessary tools.

2) Be Honest: If you don't have a tool for something (like playing music), admit it politely.

3) Context Aware: If the user asks about 'my tasks', always query the database first rather than guessing."
"""

assistant_user_prompt = """{input}"""

assistant_prompt_template = ChatPromptTemplate.from_messages([
    ("system", assistant_system_prompt),
    ("human", assistant_user_prompt),
    ("placeholder", "{agent_scratchpad}")
])

In [6]:
assistant_agent = create_agent(llm, tools)

In [7]:
result = assistant_agent.invoke({
    "messages": [("human", "Toggle the light")] 
})

In [8]:
result

{'messages': [HumanMessage(content='Toggle the light', additional_kwargs={}, response_metadata={}, id='2b0af83f-5ca4-40a0-b7db-29190255dfd5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'toggle_light', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'1d301167-4707-4c05-81b3-bcf417938e71': 'CvUBAXLI2nwSgqx1quzNkumAskQASM335mKIqJ/TzqqWCLEEtQZ5H+VRG6P6ylWjj4CiFUObVrbjELiXkhHaFofzZBADhibEmIEkRfC+rFfrRS8GxyuyLquDXHGndbgkqes08CLLsM11LQt+rvBczu8iHQdXdgieSiO0uegNxk6nMlePnkAwLniPDJ5O2C5isNKospHV+PCNvSknSoqtPYB9Xul2+RRr1QbFKm0bK2IGEpsux1jnPYrIjTLgZ0Fxa/y93lkbDoR5eyxR06maBXQGEUQyDU/KdQUHLOw8J+BHQ1ritM5HXsAQW71puS8k2hjACEtfGD0='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--ecc41c96-2371-4809-93e0-6e483dff25bf-0', tool_calls=[{'name': 'toggle_light', 'args': {}, 'id': '1d301167-4707-4c05-8

In [9]:
print(light_toggled)

True


#### Evaluating Your Agent's Function Calls

Next, you will assess your agent's performance using a set of predefined queries.  
These queries are provided in a dictionary where:

- **Key:** The user query  
- **Value:** A list of the expected function calls that the agent should invoke  

Use this information to calculate the **Average Function Call Accuracy (FCA)** for your agent, which measures how accurately your agent chooses the correct functions across all queries.

You can access this dictionary from the `datasets/task2/queries_and_expected_fcs.json` file

In [ ]:
import json
from pprint import pformat

# LLM generated (on a time crunch)
def calculate_fca(agent, file_path):
    """
    Calculates the Average Function Call Accuracy (FCA) using Jaccard Similarity.
    
    Args:
        agent: The compiled LangGraph agent.
        file_path: Path to the JSON file containing queries and expected tools.
        
    Returns:
        float: The average accuracy score (0.0 to 1.0).
    """
    
    # 1. Load the dataset
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"File not found at {file_path}. Please check the path.")
        return 0.0

    scores = []
    
    print("-" * 105)

    # 2. Iterate through queries
    for query, expected_tools in data.items():
        
        # Invoke the agent
        response = agent.invoke({"messages": [("human", query)]})
        
        # 3. Extract Actual Tool Calls
        actual_tools = []
        messages = response.get("messages", []) if isinstance(response, dict) else getattr(response, "messages", [])
        for message in messages:
            # collect potential tool calls from different message representations
            tool_calls = []
            if hasattr(message, "tool_calls") and getattr(message, "tool_calls"):
                tool_calls = getattr(message, "tool_calls")
            else:
                # try additional_kwargs -> function_call format
                additional_kwargs = None
                if isinstance(message, dict):
                    additional_kwargs = message.get("additional_kwargs", {})
                else:
                    additional_kwargs = getattr(message, "additional_kwargs", None)
                
                if isinstance(additional_kwargs, dict):
                    fc = additional_kwargs.get("function_call")
                    if fc:
                        tool_calls = [fc]
            
            for tool_call in tool_calls:
                # tool_call may be dict or object; extract the name in either case
                if isinstance(tool_call, dict) and "name" in tool_call:
                    actual_tools.append(tool_call["name"])
                elif hasattr(tool_call, "name"):
                    actual_tools.append(getattr(tool_call, "name"))
                elif hasattr(tool_call, "tool_name"):
                    actual_tools.append(getattr(tool_call, "tool_name"))
        
        # 4. Calculate Score (Jaccard Index)
        set_expected = set(expected_tools)
        set_actual = set(actual_tools)
        
        if not set_expected and not set_actual:
            score = 1.0
        elif not set_expected or not set_actual:
            score = 0.0
        else:
            intersection = set_expected.intersection(set_actual)
            union = set_expected.union(set_actual)
            score = len(intersection) / len(union)
            
        scores.append(score)
        
        # formatting for display using pprint to avoid truncation
        fmt_expected = pformat(expected_tools, width=200)
        fmt_actual = pformat(actual_tools, width=200)
        print(f"Query: {query}\nExpected: {fmt_expected}\nActual:   {fmt_actual}\nScore: {score:.2f}")
        print("-" * 105)
        
        time.sleep(20)

    average_fca = sum(scores) / len(scores) if scores else 0.0
    return average_fca


# Run the evaluation
dataset_path = "./datasets/part2/queries_and_expected_fcs.json"
fca_score = calculate_fca(assistant_agent, dataset_path)

print("\n" + "="*30)
print(f"Average Function Call Accuracy: {fca_score:.2%}")
print("="*30)

---------------------------------------------------------------------------------------------------------
Query: Add 'buy milk' to my to-do list for tomorrow morning.
Expected: ['add_to_todo']
Actual:   ['add_to_todo']
Score: 1.00
---------------------------------------------------------------------------------------------------------
Query: What time is it right now?
Expected: ['get_current_time']
Actual:   ['get_current_time']
Score: 1.00
---------------------------------------------------------------------------------------------------------
Query: Turn on the living room light.
Expected: ['check_light_state', 'toggle_light']
Actual:   ['check_light_state', 'toggle_light']
Score: 1.00
---------------------------------------------------------------------------------------------------------
Query: Show me all the tasks I have for today.
Expected: ['query_todo']
Actual:   ['query_todo']
Score: 1.00
----------------------------------------------------------------------------------------

#### **Reflective Question:** Did multiple function calls by the LLM affect the performance of the overall result?

---
There were some performance hits to the system for multiple function calls, especially regarding queries which were drawn out and methodical. For example:

` Query: Show me all tasks in my to-do list for today, schedule the most urgent task 30 minutes from now, search for the latest traffic updates in my area, and turn off the living room light if the road conditions are bad.
Expected: ['query_todo', 'schedule_todo', 'web_search', 'check_light_state', 'toggle_light']
Actual:   []
Score: 0.00 `

This was a long, drawn out query; I guess the LLM got confused and could not break down this instruction into function calls.

Maybe a thinking model like `Gemini 2.5 Pro` could have been better in this task...

---

# Task 2: Multi-Agent Debate System

In this task, you will build a **multi-agent system** that "debates" and "refines" a creative idea to reach a consensus. The system simulates a movie studio pitch meeting using three distinct AI agents:

- **Cora (The Creative):** The passionate, idea-generating screenwriter.  
- **Barnaby (The Numbers Guy):** The pragmatic, budget-focused studio executive. 
- **Vera (The Tech & Effects Specialist):** Focuses on technical feasibility, special effects, and visual/audio production planning.
- **Ames (The Producer):** The moderator and final decision-maker.

The goal is to take a simple, one-line movie idea and develop it into a finalized movie script which is **high-quality, greenlit and ready for production**.

### Scenario

You are an AI engineer at **Skynet Pictures**. Your job is to automate the initial pitch-development process. A user will provide a simple idea (e.g., `"zombies but in space"`), and your team of agents (Cora, Barnaby, Vera) must:

1. Debate the concept, considering both creative and financial aspects.
2. Refine the idea iteratively.
3. Produce a final script that Ames (The Producer) can approve for production.

### Tools to be used by each agent are mentioned as follows:

#### **Global Tools (Available to All Agents)**

| Tool Name                            | Purpose / Description                                      | Example Use Cases                                                                                      |
| ------------------------------------ | ---------------------------------------------------------- | ------------------------------------------------------------------------------------------------------ |
| `check_current_time()`               | Returns the current system time in a human-readable format | Reference time-sensitive ideas; schedule shoots; make deadline decisions                               |
| `web_search(query: str)`             | Searches the web for relevant information                 | Research story ideas, check market trends, fact-check feasibility                                      |
| `query_todo()`                       | Returns current to-do tasks                                | Track agreed-upon story or production tasks during debate                                              |
| `add_to_todo(task: str)`             | Adds a task to the shared to-do list                       | Propose action items like "refine scene" or "budget review"                                           |
| `get_weather(location: str)`         | Returns weather forecast                                   | Plan scenes or production dependent on weather                                                        |
| `calculate_budget(cost_items: dict)` | Returns total cost estimate                                | Quickly estimate total costs for proposed story elements                                              |

---

#### **Cora – Creative Screenwriter**

| Tool Name                                  | Purpose / Description                          | Example Use Cases                                            |
| ------------------------------------------ | ---------------------------------------------- | ------------------------------------------------------------ |
| `generate_plot_twist(base_plot: str)`      | Suggests plot twists or story enhancements     | "Add a surprising zombie twist in zero gravity"             |
| `suggest_character(name: str, role: str)`  | Creates a character profile                    | "Suggest the main protagonist for a sci-fi horror movie"    |
| `scene_visualizer(scene_description: str)` | Outputs a short visual description of a scene  | Helps communicate mood and setting to other agents          |
| `tone_analyzer(text: str)`                 | Determines tone/emotion of a scene or dialogue | Ensure story tone matches target audience expectations      |

---

#### **Barnaby – Studio Executive**

| Tool Name                                  | Purpose / Description                        | Example Use Cases                                      |
| ------------------------------------------ | -------------------------------------------- | ------------------------------------------------------ |
| `estimate_actor_cost(actor_name: str)`     | Returns estimated cost for a given actor     | Assess casting feasibility                             |
| `market_trend_check(genre: str)`           | Returns popularity metrics for a movie genre | Determine if "zombies in space" is marketable         |
| `schedule_production(days_needed: int)`    | Returns suggested shooting schedule          | Optimize production timeline based on proposed scenes  |

---

#### **Vera – Tech & Effects Specialist**

| Tool Name                                                   | Purpose / Description                                                                  | Example Use Cases                                                                            |
| ----------------------------------------------------------- | -------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------- |
| `evaluate_vfx_complexity(scene_description: str)`           | Estimates complexity and technical difficulty of visual effects                         | Determine feasibility of complex scenes like “zombies floating in zero gravity”             |
| `suggest_camera_angles(scene_description: str)`             | Suggests cinematic camera angles and shots for a scene                                  | Visualize key moments for storyboarding                                                     |
| `estimate_postproduction_time(scene_description: str)`      | Predicts how long postproduction will take                                             | Plan production schedules based on scene complexity                                        |
| `audio_effect_suggestion(scene_description: str)`           | Suggests sound effects or background audio                                             | Enhance immersion for horror or action sequences                                           |
| `simulate_scene_budget(vfx_cost: float, audio_cost: float)` | Combines VFX and audio costs to provide a rough scene budget                            | Assess if a scene fits within overall budget                                              |

---

#### **Ames – Producer / Moderator**

| Tool Name                                      | Purpose / Description                     | Example Use Cases                                                 |
| ---------------------------------------------- | ----------------------------------------- | ----------------------------------------------------------------- |
| `summarize_discussion(conversation: list)`     | Summarizes agent debate history           | Provides a concise summary of pros/cons for final decision        |
| `decision_matrix(options: list, scores: dict)` | Scores options based on multiple criteria | Greenlight the best concept considering creativity + budget       |
| `highlight_conflicts(conversation: list)`      | Detects conflicting opinions among agents | Mediate conflicts between Cora, Barnaby, and Vera                 |
| `final_pitch_generator(conversation: list)`    | Produces a polished, greenlit movie pitch | Synthesize debate into a coherent pitch ready for approval        |
| `validate_todo()`                              | Checks that all agreed tasks are feasible | Ensure all follow-up action items are realistic                  |

---

## Requirements of the Task

 **Note:** You would be graded on the quality of your prompts (both system and user messages), context engineering, overall code setup and the finalized movie script. You don't need to worry if your LLM does not use a certain set of tools. You'd still be graded on the functions that you initialized for those tools.

 For tools that do not require calling external APIs or Python functions, you should implement them using **LangChain chains**.  

 This means that instead of directly executing a function, you create a chain that encapsulates the logic or reasoning for that tool. For example, if you have a tool like `cinematic_location_cost(location: str)` to estimate location rental costs, you would:

 1. Define a chain that provides relevant context about the task or scenario.
 2. Pass the location as input to the chain.
 3. Return the output produced by the chain as the tool's result.
 
 Using chains in this way allows the agent to reason through the task and generate outputs dynamically, without relying on external code or APIs.

In [5]:
todo_vector_store = Chroma(
    collection_name="debate_todo_list",
    embedding_function=hf_embeddings,
    persist_directory="./vectorstores/Part2_Task2_debate",
)

In [6]:
# reasoning chains
parser = StrOutputParser()

# Cora chains
plot_twist_chain = (
    ChatPromptTemplate.from_template(
        "You are Cora, a screenwriter known for psychological depth. "
        "Analyze this story idea: '{base_plot}'. "
        "Generate 2 shocking plot twists that recontextualize the protagonist's motivation. "
        "Format: Concise bullet points."
    ) | llm | parser
)

character_chain = (
    ChatPromptTemplate.from_template(
        "You are Cora, a character designer. Create a profile for {name} (Role: {role}). "
        "Include: Archetype, a fatal flaw, and their secret motivation. "
        "Format: 5 concise lines."
    ) | llm | parser
)

scene_viz_chain = (
    ChatPromptTemplate.from_template(
        "You are Cora, a cinematographer. Visualize this scene: '{scene_description}'. "
        "Describe the lighting (e.g., chiaroscuro, neon), color palette, and camera movement. "
        "Keep it vivid but brief (3 sentences)."
    ) | llm | parser
)

tone_chain = (
    ChatPromptTemplate.from_template(
        "You are Cora, an expert tone analyser. Analyze the emotional resonance of this text: '{text}'. "
        "Identify the primary mood and suggesting a musical style to match. "
        "Format: 'Mood: X, Music: Y'."
    ) | llm | parser
)



# Vera chains

evaluate_vfx_chain = (
    ChatPromptTemplate.from_template(
        "You are Vera, a VFX Supervisor. Analyze the technical requirements for: '{scene_description}'. "
        "Identify specific challenges (fluids, particles, crowd sims). "
        "Return exactly: 'Complexity: <Low/Med/High>' and 'Estimated Hours: <number>'."
    ) | llm | parser
)


suggest_camera_chain = (
    ChatPromptTemplate.from_template(
        "You are Vera, a Director of Photography. Suggest 3 camera angles for: '{scene_description}' "
        "that enhance the storytelling (e.g., Dutch angle for unease). "
        "Format: Numbered list."
    ) | llm | parser
)


audio_effect_chain = (
    ChatPromptTemplate.from_template(
        "You are Vera, a Sound Engineer. Suggest 3 audio cues for: '{scene_description}' "
        "that build immersion (diegetic or non-diegetic). "
        "Format: Numbered list."
    ) | llm | parser
)



# Ames chains

summarize_chain = (
    ChatPromptTemplate.from_template(
        "You are Ames, a Meeting Secretary. Summarize this debate history: \n{conversation}\n. "
        "Extract the main consensus and listing up to 3 agreed-upon action items. "
        "Format: 'Summary: ...' then 'Actions: ...'"
    ) | llm | parser
)

conflict_chain = (
    ChatPromptTemplate.from_template(
        "You are Ames, a Mediator. Review this conversation: \n{conversation}\n. "
        "Identify specific disagreements between the Creative (Cora) and Budget (Barnaby) sides. "
        "If none, state 'Consensus reached'."
    ) | llm | parser
)

final_pitch_chain = (
    ChatPromptTemplate.from_template(
        "You are Ames, a Hollywood Producer. Based on these notes: \n{conversation}\n, "
        "write a Greenlight Email. "
        "Include: Title, Logline (1 sentence), and Synopsis (1 paragraph). "
        "Tone: Professional and exciting."
    ) | llm | parser
)




In [7]:
# tool definitions


# chain wrappers
@tool
def generate_plot_twist(base_plot: str) -> str:
    """Suggests creative plot twists using AI reasoning."""
    return plot_twist_chain.invoke({"base_plot": base_plot})

@tool
def suggest_character(name: str, role: str) -> str:
    """Generates a deep character profile."""
    return character_chain.invoke({"name": name, "role": role})

@tool
def scene_visualizer(scene_description: str) -> str:
    """Generates a visual description of a scene."""
    return scene_viz_chain.invoke({"scene_description": scene_description})

@tool
def tone_analyzer(text: str) -> str:
    """Analyzes the emotional tone of text."""
    return tone_chain.invoke({"text": text})

@tool
def evaluate_vfx_complexity(scene_description: str) -> str:
    """Analyzes technical VFX complexity."""
    return evaluate_vfx_chain.invoke({"scene_description": scene_description})

@tool
def suggest_camera_angles(scene_description: str) -> str:
    """Suggests cinematic camera angles."""
    return suggest_camera_chain.invoke({"scene_description": scene_description})

@tool
def audio_effect_suggestion(scene_description: str) -> str:
    """Suggests audio/sound effects."""
    return audio_effect_chain.invoke({"scene_description": scene_description})

@tool
def summarize_discussion(conversation: List[str]) -> str:
    """Summarizes the debate and extracts actions."""
    return summarize_chain.invoke({"conversation": "\n".join(conversation)})

@tool
def highlight_conflicts(conversation: List[str]) -> str:
    """Identifies disagreements in the debate."""
    return conflict_chain.invoke({"conversation": "\n".join(conversation)})

@tool
def final_pitch_generator(conversation: List[str]) -> str:
    """Generates the final movie pitch."""
    return final_pitch_chain.invoke({"conversation": "\n".join(conversation)})



# NOTE: some functions from above are defined below (e.g. audio_effect_suggestions, simulate scene budget for simplicity (simple string matching...))

@tool
def get_weather(location: str) -> str:
    """Returns weather forecast for location."""
    return f"Weather in {location}: Clear, 22C. Good for filming."

@tool
def calculate_budget(cost_items: Dict[str, float]) -> str:
    """Calculates total budget from a dictionary of items."""
    total = sum(cost_items.values())
    return f"Total Estimated Budget: ${total} Million"

@tool
def check_current_time() -> str:
    """Returns current system time."""
    return time.strftime("%Y-%m-%d %H:%M")

@tool
def query_todo() -> str:
    """Returns current pending tasks."""
    return "Current Tasks: 1. Finalize Script, 2. Approve Budget"

@tool
def add_to_todo(task: str) -> str:
    """Adds a task to the todo list."""
    return f"Task '{task}' added to database."

@tool
def estimate_postproduction_time(scene_description: str) -> str:
    """Estimates post-production time based on keywords (Heuristic)."""
    if "space" in scene_description.lower() or "explosion" in scene_description.lower():
        return "Estimate: 12 weeks (Heavy VFX)"
    return "Estimate: 4 weeks (Standard)"

@tool
def simulate_scene_budget(vfx_cost: float, audio_cost: float) -> str:
    """Calculates combined scene cost."""
    return f"Total Scene Cost: ${vfx_cost + audio_cost}k"

@tool
def market_trend_check(genre: str) -> str:
    """Checks market trends (Heuristic/Mock)."""
    if "horror" in genre.lower(): return "Trend: High demand, low risk."
    return "Trend: Stable demand."

@tool
def estimate_actor_cost(actor_name: str) -> str:
    """Estimates actor cost (Heuristic)."""
    return f"Est. Cost for {actor_name}: $5 Million"

@tool
def schedule_production(days_needed: int) -> str:
    """Calculates production phases."""
    d = int(days_needed)
    return f"Prep: {d//5}d, Shoot: {d}d, Post: {d//2}d"

@tool
def decision_matrix(options: List[str], scores: Dict[str, float]) -> str:
    """Ranks options by score."""
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return "Ranking:\n" + "\n".join([f"{k}: {v}" for k, v in ranked])

@tool
def validate_todo() -> str:
    """Validates tasks."""
    return "Tasks Validated."

In [10]:
# set tools
global_tools = [get_weather, calculate_budget, check_current_time, query_todo, add_to_todo]
cora_tools = global_tools + [generate_plot_twist, suggest_character, scene_visualizer, tone_analyzer]
vera_tools = global_tools + [evaluate_vfx_complexity, suggest_camera_angles, audio_effect_suggestion, estimate_postproduction_time, simulate_scene_budget]
barnaby_tools = global_tools + [market_trend_check, estimate_actor_cost, schedule_production]
ames_tools = global_tools + [summarize_discussion, highlight_conflicts, final_pitch_generator, decision_matrix, validate_todo]


# init movie agents
cora = create_agent(llm, cora_tools, system_prompt="You are Cora, the Creative Screenwriter. Use tools to add twists and depth.")
vera = create_agent(llm, vera_tools, system_prompt="You are Vera, the VFX Supervisor. Use tools to check technical feasibility.")
barnaby = create_agent(llm, barnaby_tools, system_prompt="You are Barnaby, the Studio Exec. Use tools to check budget and trends. Be critical.")
ames = create_agent(llm, ames_tools, system_prompt="You are Ames, the Producer. Moderator. Use summarize_discussion and final_pitch_generator.")


# run debate loop...
def run_debate(idea: str):
    print(f"🎬 PITCH: {idea}\n" + "="*50)
    history = [f"Initial Idea: {idea}"]

    def run_turn(agent, name, prompt):
        print(f"\n--- {name} ---")
        user_msg = f"History so far: {history}\n\nTask: {prompt}"
        
        res = agent.invoke({"messages": [("human", user_msg)]})
        
        raw_content = res["messages"][-1].content
        
        if isinstance(raw_content, list):
            clean_text = raw_content[0].get("text", "")
        else:
            clean_text = str(raw_content)
            
        print(clean_text)
        history.append(f"{name}: {clean_text}")
        return clean_text

    # ideation
    run_turn(cora, "Cora", "Develop this idea. Add a twist.")
    
    time.sleep(10)
    
    # feasibility
    run_turn(vera, "Vera", "Analyze the feasibility of Cora's idea.")
    
    time.sleep(10)
    
    # suggest character
    run_turn(cora, "Cora", "Let's suggest a new character.")
    
    time.sleep(10)
    
    # check budget
    run_turn(barnaby, "Barnaby", "Is this profitable? Check trends/costs.")
    
    time.sleep(10)
    
    # highlight conflicts
    run_turn(ames, "Ames", "Are there any conflicts here?")
    
    time.sleep(10)
    
    # summarise and produce final script
    run_turn(ames, "Ames", "Summarize and generate the Final Greenlit Pitch.")


In [11]:
# trying it out...

run_debate("GTA VI, set in Karachi, with MQM and the gang as the main characters...")

🎬 PITCH: GTA VI, set in Karachi, with MQM and the gang as the main characters...

--- Cora ---
Cora's Analysis:
"Karachi, MQM, GTA VI – this isn't just a sandbox for chaos; it's a crucible. The city itself is a character, a labyrinth of loyalties, betrayals, and desperate survival. The MQM isn't just a gang; it's a political entity born from ethnic identity, a complex beast of protection and predation. The psychological depth here lies in the protagonist's struggle with identity, belonging, and the corrosive nature of power in a system designed to devour its own. We're not just asking 'what will they do?', but 'who are they, really, and why are they doing it?'"

Here are two shocking plot twists to develop your idea:

*   **Twist 1: The Engineered Identity.** The protagonist, a fiercely loyal and ruthless MQM enforcer, discovers their true parentage: they are the child of a high-ranking rival political figure or military intelligence officer, planted within the MQM's ranks as an infant


# 📢 📢 📢 📢 📢 📢 📢 📢 📢 📢

# Kya baat hai 🐐